# Food Dude
### Jaiden Gerig, Oron Hazi, Justin Katz, Kyle Wilson

## Importing Our Data
Luckily this is all in a CSV file so we can grab it easilty with Pandas

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

foods = pd.read_csv('ABBREV.csv', index_col=0)
foods.head()

,Shrt_Desc,Water_(g),Energ_Kcal,Protein_(g),Lipid_Tot_(g),Ash_(g),Carbohydrt_(g),Fiber_TD_(g),Sugar_Tot_(g),Calcium_(mg),...,Vit_K_(�g),FA_Sat_(g),FA_Mono_(g),FA_Poly_(g),Cholestrl_(mg),GmWt_1,GmWt_Desc1,GmWt_2,GmWt_Desc2,Refuse_Pct
NDB_No,,,,,,,,,,,,,,,,,,,,,
1001,"BUTTER,WITH SALT",15.87,717,0.85,81.11,2.11,0.06,0.0,0.06,24.0,...,7.0,51.368,21.021,3.043,215.0,5.00,"1 pat, (1"" sq, 1/3"" high)",14.2,1 tbsp,0.0
1002,"BUTTER,WHIPPED,W/ SALT",16.72,718,0.49,78.30,1.62,2.87,0.0,0.06,23.0,...,4.6,45.390,19.874,3.331,225.0,3.80,"1 pat, (1"" sq, 1/3"" high)",9.4,1 tbsp,0.0
1003,"BUTTER OIL,ANHYDROUS",0.24,876,0.28,99.48,0.00,0.00,0.0,0.00,4.0,...,8.6,61.924,28.732,3.694,256.0,12.80,1 tbsp,205.0,1 cup,0.0
1004,"CHEESE,BLUE",42.41,353,21.40,28.74,5.11,2.34,0.0,0.50,528.0,...,2.4,18.669,7.778,0.800,75.0,28.35,1 oz,17.0,1 cubic inch,0.0
1005,"CHEESE,BRICK",41.11,371,23.24,29.68,3.18,2.79,0.0,0.51,674.0,...,2.5,18.764,8.598,0.784,94.0,132.00,"1 cup, diced",113.0,"1 cup, shredded",0.0


## Washing Our Food
to start off we only wanted to look at the main nutritional values (Protein,Fat,Sodium,Etc.) of each food

In [2]:
#Rename some of our columns to something a bit easier on the eyes
foods = foods.rename(index=str,columns={'Shrt_Desc':'Name','Protein_(g)':'Protein (g)','Lipid_Tot_(g)':'Total Fat(g)','Cholestrl_(mg)':'Cholesterol (mg)',
               'FA_Sat_(g)':'Saturated Fat (g)','Sodium_(mg)':'Sodium (mg)','Potassium_(mg)':'Potassium (mg)',
               'Carbohydrt_(g)':'Carbohydrates (g)','Fiber_TD_(g)':'Fiber (g)','GmWt_1':'Weight (g)'})
# Look at a specific subset of nutrients
nutrients = ['Name','Protein (g)','Total Fat(g)','Cholesterol (mg)',
               'Saturated Fat (g)','Sodium (mg)','Potassium (mg)',
               'Carbohydrates (g)','Fiber (g)','Weight (g)']
foods = foods[nutrients]
foods = foods.fillna(0)
# Get rid of foods we dont have serving sizes for
foods = foods[foods.apply(lambda x:x['Weight (g)'] > 0, axis=1)]
foods.head()

,Name,Protein (g),Total Fat(g),Cholesterol (mg),Saturated Fat (g),Sodium (mg),Potassium (mg),Carbohydrates (g),Fiber (g),Weight (g)
NDB_No,,,,,,,,,,
1001,"BUTTER,WITH SALT",0.85,81.11,215.0,51.368,643.0,24.0,0.06,0.0,5.00
1002,"BUTTER,WHIPPED,W/ SALT",0.49,78.30,225.0,45.390,583.0,41.0,2.87,0.0,3.80
1003,"BUTTER OIL,ANHYDROUS",0.28,99.48,256.0,61.924,2.0,5.0,0.00,0.0,12.80
1004,"CHEESE,BLUE",21.40,28.74,75.0,18.669,1146.0,256.0,2.34,0.0,28.35
1005,"CHEESE,BRICK",23.24,29.68,94.0,18.764,560.0,136.0,2.79,0.0,132.00


We wanted to look the nutritional value of each food regardless of serving size, so we normalized them to 1 gram

In [3]:
# Normalizing all our foods to 1 g
def normalizeNutrients(x):
    ratio = x['Weight (g)']
    for nutrient in nutrients:
        if(type(x[nutrient]) is str):
            continue
        x[nutrient] = x[nutrient]/ratio
    return x
foods = foods.apply(normalizeNutrients,axis=1)
foods.head()

,Name,Protein (g),Total Fat(g),Cholesterol (mg),Saturated Fat (g),Sodium (mg),Potassium (mg),Carbohydrates (g),Fiber (g),Weight (g)
NDB_No,,,,,,,,,,
1001,"BUTTER,WITH SALT",0.170000,16.222000,43.000000,10.273600,128.600000,4.800000,0.012000,0.0,1.0
1002,"BUTTER,WHIPPED,W/ SALT",0.128947,20.605263,59.210526,11.944737,153.421053,10.789474,0.755263,0.0,1.0
1003,"BUTTER OIL,ANHYDROUS",0.021875,7.771875,20.000000,4.837813,0.156250,0.390625,0.000000,0.0,1.0
1004,"CHEESE,BLUE",0.754850,1.013757,2.645503,0.658519,40.423280,9.029982,0.082540,0.0,1.0
1005,"CHEESE,BRICK",0.176061,0.224848,0.712121,0.142152,4.242424,1.030303,0.021136,0.0,1.0


## Making a Meal
We started our food analysis by taking all the foods and Scaling them up to meet the demands of an average a 2000 calorie diet according to [Netrition](http://www.netrition.com/rdi_page.html).

The first step we took was to exclude foods that didnt contain all the nutrients we were looking at

In [4]:
# We only want foods that have a chance to sustain our needs
def filterNutrients(x):
    for nutrient in nutrients:
        if(x[nutrient] <= 0):
            return False
    return True
foods = foods[foods.apply(filterNutrients, axis=1)]
print 'Matching Foods:',len(foods)
foods.head()

Matching Foods: 1396


,Name,Protein (g),Total Fat(g),Cholesterol (mg),Saturated Fat (g),Sodium (mg),Potassium (mg),Carbohydrates (g),Fiber (g),Weight (g)
NDB_No,,,,,,,,,,
1013,"CHEESE,COTTAGE,CRMD,W/FRUIT",0.094602,0.034071,0.115044,0.020451,3.044248,0.796460,0.040796,0.001770,1.0
1043,"CHEESE,PAST PROCESS,PIMENTO",0.158071,0.222857,0.671429,0.140450,6.535714,1.157143,0.012357,0.000714,1.0
1102,"MILK,CHOC,FLUID,COMM,WHL,W/ ADDED VIT A & VITA...",0.012680,0.013560,0.048000,0.008416,0.240000,0.668000,0.041360,0.003200,1.0
1103,"MILK,CHOC,FLUID,COMM,RED FAT",0.011960,0.007600,0.032000,0.004708,0.264000,0.676000,0.048520,0.002800,1.0
1104,"MILK,CHOC,LOWFAT,W/ ADDED VIT A & VITAMIN D",0.013840,0.004000,0.020000,0.002336,0.260000,0.688000,0.039440,0.000400,1.0


Nice, That leaves us with almost 1400 foods to look at, not too shabby
So let's take these foods and scale them up to see how many grams of each we would have to consume to fulfill our daily nutrition requirements

In [28]:
# http://www.netrition.com/rdi_page.html
recommended = [-1,50,65,300,20,2400,3500,300,25,-1]
def findSatisfyingWeight(food):
    for x in range(0,len(nutrients)):
        nutrient = nutrients[x]
        rec = recommended[x]
        if(rec == -1 or food[nutrient] >= rec):
            continue
        ratio = rec/food[nutrient]
        for y in nutrients:
            if(type(food[y]) is str):
                continue
            food[y] = food[y]*ratio
    return food   
weighted_foods = foods.apply(findSatisfyingWeight,axis=1)
display = ['Name','Weight (g)','Protein (g)','Total Fat(g)','Cholesterol (mg)',
               'Saturated Fat (g)','Sodium (mg)','Potassium (mg)',
               'Carbohydrates (g)','Fiber (g)']
weighted_foods[display].sort_values(by='Weight (g)').head(10)

,Name,Weight (g),Protein (%),Total Fat(%),Cholesterol (%),Saturated Fat (%),Sodium (%),Potassium (%),Carbohydrates (%),Fiber (g)
NDB_No,,,,,,,,,,
43260,"BEVERAGE,INST BRKFST PDR,CHOC,SUGAR-FREE,NOT R...",71.372549,456.274510,65.000000,560.784314,27.554902,9138.235294,21730.392157,522.549020,25.490196
6959,"GRAVY,INST TURKEY,DRY",83.750000,146.500000,183.250000,300.000000,61.637500,51125.000000,3825.000000,719.500000,47.500000
21422,"KFC,POPCORN CHICK",160.000000,441.750000,543.500000,1000.000000,98.850000,28500.000000,7200.000000,529.500000,25.000000
6957,"GRAVY,BROWN INST,DRY",167.500000,213.250000,296.250000,300.000000,146.650000,126325.000000,9175.000000,1494.500000,80.000000
43078,"BEVERAGE,MILKSHAKE MIX,DRY,NOT CHOC",175.000000,587.500000,65.000000,350.000000,51.475000,19500.000000,55000.000000,1322.500000,40.000000
1223,"PROTEIN SUPP,MILK BSD,MUSCLE MILK,PDR",178.378378,741.243243,277.945946,340.540541,25.183784,5335.135135,18308.108108,300.000000,115.135135
6958,"GRAVY,INST BF,DRY",182.727273,267.272727,258.545455,300.000000,133.090909,141900.000000,12272.727273,1666.363636,117.272727
6124,"GRAVY,PORK,DRY,PDR",201.000000,263.400000,258.900000,300.000000,128.700000,160680.000000,7050.000000,1907.100000,72.000000
19120,"CANDIES,MILK CHOC",212.658228,232.405063,901.063291,698.734177,562.298734,2400.000000,11301.265823,1804.556962,103.291139


Yikes, none of that seems very healthy at all

To some of these foods into perspective:

* KFC Popcorn Chicken: 160 grams = 25 pieces
* Milkshake Mix: 175 grams = 25 tablespoons
* Milk Choclate: 213 grams = 30 Bars

Let's see what happens when we look at how far over the reccomended nutritional values these foods go

In [70]:
from bokeh.io import push_notebook,show,output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.charts import Bar, output_file, show
from bokeh.models import Range1d
from bokeh.charts.operations import blend
from bokeh import palettes
def findOverages(food):
    for x in range(0,len(nutrients)):
        nutrient = nutrients[x]
        rec = recommended[x]
        if(rec == -1):
            continue
        food[nutrient] -= rec
    return food   
overage_foods = weighted_foods.apply(findOverages,axis=1)
df = overage_foods.sort_values(by='Weight (g)').head(10)
a = Bar(df, 'Name', values='Protein (g)', title="Excess Protein",legend=False,width=450,continuous_range=Range1d(0,100))
b = Bar(df, 'Name', values='Total Fat(g)', title="Excess Total Fat",legend=False,width=450)
a.xaxis.axis_label = ""
b.xaxis.axis_label = ""
show(row(a,b))
a = Bar(df, 'Name', values='Cholesterol (mg)', title="Excess Cholesterol",legend=False,width=450)
b = Bar(df, 'Name', values='Saturated Fat (g)', title="Excess Saturated Fats",legend=False,width=450)
a.xaxis.axis_label = ""
b.xaxis.axis_label = ""
show(row(a,b))
a = Bar(df, 'Name', values='Sodium (mg)', title="Excess Sodium",legend=False,width=450)
b = Bar(df, 'Name', values='Potassium (mg)', title="Excess Potassium",legend=False,width=450)
a.xaxis.axis_label = ""
b.xaxis.axis_label = ""
show(row(a,b))
a = Bar(df, 'Name', values='Carbohydrates (g)', title="Excess Carbohydrates",legend=False,width=450)
b = Bar(df, 'Name', values='Fiber (g)', title="Excess Fiber",legend=False,width=450)
a.xaxis.axis_label = ""
b.xaxis.axis_label = ""
show(row(a,b))
output_notebook()

Loading BokehJS ...

So that gives us a general idea of what kind of foods have common excesses but it's hard to compare them to each-other so let's convert them to percentages of the 2000 calorie diet over 100%

In [29]:
def findPercentOverages(food):
    for x in range(0,len(nutrients)):
        nutrient = nutrients[x]
        rec = recommended[x]
        if(rec == -1):
            continue
        food[nutrient] = ((food[nutrient]/rec)*100)-100
    return food   
overage_foods = weighted_foods.apply(findPercentOverages,axis=1)
overage_foods[display].sort_values(by='Weight (g)').head(10).rename(index=str,columns={'Protein (g)':'Protein (%)','Total Fat(g)':'Total Fat(%)','Cholesterol (mg)':'Cholesterol (%)',
               'Saturated Fat (g)':'Saturated Fat (%)','Sodium (mg)':'Sodium (%)','Potassium (mg)':'Potassium (%)',
               'Carbohydrates (g)':'Carbohydrates (%)','Fiber (g)':'Fiber (g)'})

,Name,Weight (g),Protein (%),Total Fat(%),Cholesterol (%),Saturated Fat (%),Sodium (%),Potassium (%),Carbohydrates (%),Fiber (g)
NDB_No,,,,,,,,,,
43260,"BEVERAGE,INST BRKFST PDR,CHOC,SUGAR-FREE,NOT R...",71.372549,812.549020,0.000000,86.928105,37.774510,280.759804,520.868347,74.183007,1.960784
6959,"GRAVY,INST TURKEY,DRY",83.750000,193.000000,181.923077,0.000000,208.187500,2030.208333,9.285714,139.833333,90.000000
21422,"KFC,POPCORN CHICK",160.000000,783.500000,736.153846,233.333333,394.250000,1087.500000,105.714286,76.500000,0.000000
6957,"GRAVY,BROWN INST,DRY",167.500000,326.500000,355.769231,0.000000,633.250000,5163.541667,162.142857,398.166667,220.000000
43078,"BEVERAGE,MILKSHAKE MIX,DRY,NOT CHOC",175.000000,1075.000000,0.000000,16.666667,157.375000,712.500000,1471.428571,340.833333,60.000000
1223,"PROTEIN SUPP,MILK BSD,MUSCLE MILK,PDR",178.378378,1382.486486,327.609148,13.513514,25.918919,122.297297,423.088803,0.000000,360.540541
6958,"GRAVY,INST BF,DRY",182.727273,434.545455,297.762238,0.000000,565.454545,5812.500000,250.649351,455.454545,369.090909
6124,"GRAVY,PORK,DRY,PDR",201.000000,426.800000,298.307692,0.000000,543.500000,6595.000000,101.428571,535.700000,188.000000
19120,"CANDIES,MILK CHOC",212.658228,364.810127,1286.251217,132.911392,2711.493671,0.000000,222.893309,501.518987,313.164557


And now let's chart them again with a uniform scale


In [60]:
df = overage_foods.sort_values(by='Weight (g)').head(10).rename(index=str,columns={'Protein (g)':'Protein (%)','Total Fat(g)':'Total Fat(%)','Cholesterol (mg)':'Cholesterol (%)',
               'Saturated Fat (g)':'Saturated Fat (%)','Sodium (mg)':'Sodium (%)','Potassium (mg)':'Potassium (%)',
               'Carbohydrates (g)':'Carbohydrates (%)','Fiber (g)':'Fiber (g)'})
a = Bar(df, label='vars',group='Name', 
        values=blend('Protein (%)', 'Total Fat(%)','Cholesterol (%)',
                     'Saturated Fat (%)','Sodium (%)','Potassium (%)',
                     'Carbohydrates (%)','Fiber (%)',name='values', labels_name='vars'),
        title="Excess Nutrients (% above recommended daily intake)",width=900)
a.xaxis.axis_label = ""
a.yaxis.axis_label = "% above reccomended daily intake"
show(a)
output_notebook()

Loading BokehJS ...

Holy Guacamole! Look at that sodium!

It looks like potassium and sodium are crazy high compared to the other nutrients, which is most likely because they're the only nutrients measured in milligrams instead of grams, so let's see what the graph looks like without them so we can get a better understanding of the other nutrients

In [79]:
df = overage_foods.sort_values(by='Weight (g)').head(10).rename(index=str,columns={'Protein (g)':'Protein (%)','Total Fat(g)':'Total Fat(%)','Cholesterol (mg)':'Cholesterol (%)',
               'Saturated Fat (g)':'Saturated Fat (%)','Sodium (mg)':'Sodium (%)','Potassium (mg)':'Potassium (%)',
               'Carbohydrates (g)':'Carbohydrates (%)','Fiber (g)':'Fiber (g)'})
a = Bar(df, label='vars',group='Name', 
        values=blend('Protein (%)', 'Total Fat(%)','Cholesterol (%)',
                     'Saturated Fat (%)',
                     'Carbohydrates (%)','Fiber (%)',name='values', labels_name='vars'),
        title="Excess Nutrients (% above recommended daily intake) (Excluding Sodium & Potassium)",width=900,height=1000,palette=palettes.BrBG11)
a.xaxis.axis_label = ""
a.yaxis.axis_label = "% above reccomended daily intake"
show(a)
output_notebook()

Loading BokehJS ...

So overall, it appears that these foods are providing mainly carbs and sodium at the expense of other nutrients 